Item referenced:

    https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
    https://github.com/keras-team/keras/issues/3296

In [17]:
# importing all the needed libraries here
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Dense, Input

In [18]:
# path to the model weights files.
weights_path = 'weights/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 496
nb_validation_samples = 128
epochs = 10
batch_size = 16

In [19]:
# build the VGG16 network
vgg16_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')
print("Model output shape is: ", vgg16_model.output_shape)

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

Model loaded.
('Model output shape is: ', (None, 4, 4, 512))


In [20]:
model = Sequential() #new model
for layer in vgg16_model.layers: 
    model.add(layer)

# add the model on top of the convolutional base
model.add(top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [21]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
        shuffle=False)

print(len(train_generator.filenames))
print(train_generator.class_indices)
print(len(train_generator.class_indices))

Found 509 images belonging to 32 classes.


In [22]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs)
    #validation_data=validation_generator,
    #nb_val_samples=nb_validation_samples)
model.save_weights('vgg_leaf_identification_v0.1.h5')
model.save('vgg_leaf_identification_model_v0.1.h5')
print("Model and weights saved")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, steps_per_epoch=31)`
  """


Epoch 1/10


ValueError: Error when checking model target: expected sequential_5 to have shape (None, 1) but got array with shape (16, 32)

In [8]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=4,
    class_mode='categorical')

Found 128 images belonging to 32 classes.


In [14]:
len_validate = len(validation_generator.filenames)

In [10]:
final_model = load_model('vgg_leaf_identification_model_v0.1.h5')

In [11]:
preds = final_model.predict_generator(validation_generator, len_validate)

In [13]:
validation_generator